<a href="https://colab.research.google.com/github/Danh1905/Scalable-Project/blob/main/scalableProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To upload a file from your local system, you can use `google.colab.files.upload()`:

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving customer_churn.csv to customer_churn.csv
User uploaded file "customer_churn.csv" with length 115479 bytes


Install pyspark


In [ ]:
!pip install pyspark py4j

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Read CSV").getOrCreate()
churn_rate = spark.read.csv("/content/customer_churn.csv", header=True, inferSchema=True)
churn_rate.printSchema()


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



Check if there're any null value

In [15]:
from pyspark.sql.functions import col, sum, when

def count_nulls(df):
    null_counts = df.select([
        sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in df.columns
    ])
    return null_counts

null_df = count_nulls(churn_rate)
null_df.show()


+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|    0|  0|             0|              0|    0|        0|           0|       0|      0|    0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+

